In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import corner
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
n_samples = 1000

data = np.random.uniform(size=[n_samples, 1])

In [ ]:
def define_nf():
    
    return tfd.TransformedDistribution(
        distribution=tfd.Sample(tfd.Normal(loc=0., scale=1.), sample_shape=[1]),
        bijector=tfb.MaskedAutoregressiveFlow(tfb.AutoregressiveNetwork(
            params=2,
            hidden_units=[4],
            activation='relu',
            )),
        )

In [ ]:
nf = define_nf()

In [ ]:
def define_model(nf):
    
    x = tf.keras.Input(shape=[1], dtype=tf.float32)

    return tf.keras.Model(inputs=x, outputs=nf.log_prob(x))

In [ ]:
model = define_model(nf)
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss=lambda _, log_prob: -log_prob,
    )

In [ ]:
nf.variables, model.weights

In [ ]:
xx = np.linspace(-.5, 1.5, 100)
px = np.exp(nf.log_prob(xx[:, None]))
plt.plot(xx, px);

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./saving.hdf5',
        monitor='loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True,
        save_freq='epoch',
        verbose=1,
        ),
    ]

result = model.fit(
    x=data, y=np.zeros(n_samples), epochs=10, callbacks=callbacks,
    )

In [ ]:
nf.variables, model.weights

In [ ]:
plt.plot(xx, px)

px = np.exp(nf.log_prob(xx[:, None]))
plt.plot(xx, px)

px_ = np.exp(model.predict_on_batch(xx))
plt.plot(xx, px_, ls='--');

In [ ]:
with h5py.File('./saving.hdf5', 'r') as h:
    print(h.keys())

In [ ]:
nf = define_nf()
model = define_model(nf)
nf.variables, model.weights

In [ ]:
model.load_weights('./saving.hdf5')
nf.variables, model.weights

In [ ]:
plt.plot(xx, px, lw=10)

px = np.exp(nf.log_prob(xx[:, None]))
plt.plot(xx, px, lw=5)

px_ = np.exp(model.predict_on_batch(xx))
plt.plot(xx, px_, ls='--', lw=2);